In [7]:
import nshutils as nu

nu.pretty()

In [ ]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams
    hparams.model = MC.JMPBackboneConfig.draft()
    hparams.model.graph_computer = MC.GraphComputerConfig.draft()
    hparams.model.graph_computer.pbc = True

    hparams.model.ckpt_path = Path("/mnt/shared/checkpoints/jmp-l.pt")
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.PerSplitDataConfig.draft()
    hparams.data.train = MC.OMAT24DatasetConfig.draft()
    hparams.data.train.src = Path("/mnt/datasets/salex/val/")
    hparams.data.batch_size = 1

    # Trainer hparams
    hparams.lightning_trainer_kwargs = {"fast_dev_run": True}

    hparams = hparams.finalize(strict=False)
    return hparams


model = MatterTuner(hparams()).tune()
model

Unrecognized arguments:  dict_keys(['ln', 'dropout', 'replace_scale_factors_with_ln', 'learnable_rbf', 'learnable_rbf_stds', 'unique_basis_per_layer', 'old_gaussian_implementation', 'edge_dropout'])


In [ ]:
potential = model.potential()
print(potential)

calculator = model.ase_calculator()
print(calculator)


In [ ]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(potential.predict([atoms], model.hparams.properties))

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


Predicting: |          | 0/? [00:00<?, ?it/s]

[{'energy': tensor[1] [5.390], 'forces': tensor[3, 3] n=9 x∈[-1.419, 1.253] μ=-0.010 σ=0.868 [[-0.000, -0.062, 0.278], [0.000, -1.419, 1.016], [0.000, 1.253, -1.155]], 'stresses': tensor[1, 3, 3] n=9 x∈[-1.702e-06, 0.249] μ=0.083 σ=0.090 [[[0.249, 1.669e-06, -1.702e-06], [1.669e-06, 0.100, 0.116], [-1.702e-06, 0.116, 0.164]]]}]


In [ ]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


Predicting: |          | 0/? [00:00<?, ?it/s]

5.39019775390625


In [ ]:
potential = model.potential()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = potential.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

ase.Atoms 1 energy: tensor[1] [5.390]
ase.Atoms 1 forces: tensor[3, 3] n=9 x∈[-1.419, 1.253] μ=-0.010 σ=0.868 [[-0.000, -0.062, 0.278], [0.000, -1.419, 1.016], [0.000, 1.253, -1.155]]
ase.Atoms 2 energy: tensor[1] [5.390]
ase.Atoms 2 forces: tensor[3, 3] n=9 x∈[-1.419, 1.253] μ=-0.010 σ=0.868 [[-0.000, -0.062, 0.278], [0.000, -1.419, 1.016], [0.000, 1.253, -1.155]]
